# Captain Freako's Solutions to *Typeclassopedia* Exercises
Original author: David Banas <capn.freako@gmail.com>  
Original date: September 25, 2015

Copyright (c) 2015 David Banas; all rights reserved World wide.

This [jupyter notebook](https://github.com/gibiansky/ihaskell) contains my solutions to some of the exercises posed by [Brent Yorgey](mailto:byorgey@cis.upenn.edu), in his [Typeclassopedia](https://wiki.haskell.org/Typeclassopedia) document.

## Additional Contributors
- Phil Ruffwind <rf@rufflewind.com>


## Contents <a name="contents"/>
- <a href="#3.2">Section 3.2 - Functor Instances</a>
    - <a href="#3.2.1">Ex. 3.2.1 - Functor instances for Either e and ((->) e)</a>
    - <a href="#3.2.2">Ex. 3.2.2 - Functor instances for ((,) e) and Pair</a>
    - <a href="#3.2.3">Ex. 3.2.3 - Functor instances for a tree</a>
    - <a href="#3.2.4">Ex. 3.2.4 - Non-Functor type of kind: \* -> \* </a>
    - <a href="#3.2.5">Ex. 3.2.5 - Is the composition of two Functors also a Functor?</a>
- <a href="#3.3">Section 3.3 - Functor Laws</a>
    - <a href="#3.3.1">Ex. 3.3.1 - Satisfies second Functor law, but not first.</a>
    - <a href="#3.3.2">Ex. 3.3.2 - Which laws violated by bad Functor list instance?</a>
- <a href="#4.2">Section 4.2 - Applicative Laws</a>
    - <a href="#4.2.1">Ex. 4.2.1 - Applying a pure function to an effectful argument.</a>
- <a href="#4.3">Section 4.3 - Applicative Instances</a>
    - <a href="#4.3.1">Ex. 4.3.1 - Maybe</a>
    - <a href="#4.3.2">Ex. 4.3.2 - ZipList</a>
- <a href="#4.5">Section 4.5 - Alternative Formulation (Monoidal)</a>
    - <a href="#4.5.1">Ex. 4.5.1 - pure and <*>, via unit and (**)</a>
    - <a href="#4.5.2">Ex. 4.5.2 - Applicative instances providing inverses of Monoidal functions</a>
    - <a href="#4.5.3">Ex. 4.5.3 - Equivalency of Applicative and Monoidal laws</a>
    

In [1]:
-- All language pragmas should come before any other code cells, to avoid confusing errors.
{-# LANGUAGE
 DeriveFunctor
 DatatypeContexts  -- Needs to be included, despite deprecation warning.
 ScopedTypeVariables
 #-}

import Data.Time

putStrLn "Notebook last run:"
getCurrentTime

Warning: -XDatatypeContexts is deprecated: It was widely considered a misfeature, and has been removed from the Haskell language.

Notebook last run:

2015-12-31 15:29:50.091867 UTC

## Section 3.2 - Functor Instances <a name="3.2"/>

### Ex. 3.2.1 - Functor instances for Either e and ((->) e) <a name="3.2.1"/>

In [2]:
class MyFunctor f where
    fmap' :: (a -> b) -> f a -> f b

instance MyFunctor (Either e) where
    fmap' g (Left e)  = Left e
    fmap' g (Right x) = Right (g x)

instance MyFunctor ((->) e) where
    fmap' g f = g . f

-- Testing Functor law on defined instances.
func_law x = fmap' id x == x  -- "id x" was producing an annoying error.

func_law2 x = fmap' id f x == f x
  where
    f x = 2 * x

func_law (Left "Error String")
func_law (Right 1)
func_law2 (2::Int)


True

True

True

[Back to Contents](#contents)

### Ex. 3.2.2 - Functor instances for ((,) e) and Pair <a name="3.2.2"/>

In [3]:
instance MyFunctor ((,) e) where
    fmap' g (e, a) = (e, g a)

data Pair a = Pair a a deriving (Eq, Show)

instance MyFunctor Pair where
    fmap' g (Pair x y) = Pair (g x) (g y)

func_law ("msg", 0.1)
func_law $ Pair 1 2


True

True

__Discussion, as per exercise instructions:__

  -  Similarities:
  
     - Both types/instances return a pair of values.

  -  Differences:
  
    - The type of the first member of the pair can differ from that
      of the second, in the '(,) e' case.
      
    - The mapped function, g, is only mapped over the second member
      of the pair, in the '(,) e' case.
      
    - The type of the second member of the pair is NOT included in
      the Functor instance, for either type, but the type of the
      first member IS included, for ((,) e), but NOT for Pair.


[Back to Contents](#contents)

### Ex. 3.2.3 - Functor instance for a tree <a name="3.2.3"/>

In [4]:
data ITree a = Leaf (Int -> a) 
             | Node [ITree a]

instance Eq a => Eq (ITree a) where
    Leaf g == Leaf h = g 0 == h 0
    Node xs == Node ys = and (zipWith (==) xs ys)
    Leaf _ == Node _ = False
    Node _ == Leaf _ = False

instance MyFunctor ITree where
    fmap' g (Leaf f)   = Leaf (g . f)
    fmap' g (Node its) = Node $ map (fmap' g) its

-- Test Functor identity law.
let f = Node [Leaf id, Leaf id] in
  fmap' id f == f


True

[Back to Contents](#contents)

### Ex. 3.2.4 - Non-Functor type of kind: \* -> \* <a name="3.2.4"/>

In [5]:
-- data NoFunc a = NoFunc a deriving (Functor)  -- Compiles without error.
data (Ord a) => NoFunc a = NoFunc a deriving (Functor)


Can't make a derived instance of ‘Functor interactive:Ghci150.NoFunc’: Data type ‘interactive:Ghci150.NoFunc’ must not have a class context (Ord a)
In the data declaration for ‘interactive:Ghci150.NoFunc’

[Back to Contents](#contents)

### Ex. 3.2.5 - Is the composition of two Functors also a Functor? <a name="3.2.5"/>

In [16]:
-- Yes: The following code compiles without error.
data Func1 a = Func1 a deriving (Functor)
data Func2 b = Func2 b deriving (Functor)
data CompFunc b = CompFunc (Func1 (Func2 b)) deriving (Functor)


[Back to Contents](#contents)

## Section 3.3 - Functor Laws <a name="3.3"/>

### Ex. 3.3.1 - Satisfies second Functor law, but not first. <a name="3.3.1"/>
I was pointed to the following solution, by Phil Ruffwind <rf@rufflewind.com>:

[Stack Overflow Solution](http://stackoverflow.com/questions/13539335/haskell-first-functor-law-from-second/13539433#13539433)


In [7]:
data Break a = Yes | No deriving (Eq)

instance MyFunctor Break where
   fmap' f _ = No

-- Test Functor id law.
fmap' id Yes == Yes

-- Test Functor composition law.
let g = id
    h = const Yes
  in fmap' (g . h) Yes == (fmap' g . fmap' h) Yes


False

True

[Back to Contents](#contents)

### Ex. 3.3.2 - Which laws violated by bad Functor list instance? <a name="3.3.2"/>
Both, because each application of fmap doubles the size of the list.

In [8]:
-- Evil Functor instance
instance MyFunctor [] where
  fmap' _ [] = []
  fmap' g (x:xs) = g x : g x : fmap g xs

-- Testing Functor identity law.
fmap' id [1,2,3] == [1,2,3]

-- Testing Functor composition law.
fmap' ((+ 1) . (* 2)) [1,2,3] == (fmap' (+ 1) . fmap' (* 2)) [1,2,3]


False

False

[Back to Contents](#contents)

## Section 4.2 - Applicative Laws <a name="4.2"/>

### Ex. 4.2.1 - Applying a pure function to an effectful argument. <a name="4.2.1"/>

__To prove:__
```
pure f <*> x = pure (flip ($)) <*> x <*> pure f
```

__Available Laws/Properties:__
```
pure f <*> x      = fmap f x            = f <$> x                  (1st Applicative Law)
f <*> pure x      = pure ($ x) <*> f    = pure (flip ($) x) <*> f  (interchange)
pure f <*> pure x = pure (f x)                                     (homomorphism)
fmap h (fmap g f) = (fmap h . fmap g) f = fmap (h . g) f           (2nd Functor Law)
```

__Useful Relationships:__
```
f     = \x -> f x =      ($) f = (f $)                             (function/lambda definition)
($ x) = \f -> f x = flip ($) x                                     ("($ x)" syntax definition)`
```

__Proof:__
```
pure (flip ($)) <*> x <*> pure f       =  (associativity)
(pure (flip ($)) <*> x) <*> pure f     =  (interchange)
pure ($ f) <*> (pure (flip ($)) <*> x) =  (1st Applicative Law)
fmap (flip ($) f) (fmap (flip ($)) x)  =  (g = flip ($))
fmap (g f) (fmap g x)                  =  (Second Functor Law)
fmap (g f . g) x                       =  (lemma 1, below)
fmap f x                               =  (1st Applicative Law)
pure f <*> x
```
&#9633;

__Lemma 1:__
```
(flip ($) g) . flip ($) == g
```

__Types:__
```
g                       :: a -> b
flip ($)                :: a1 -> ((a1 -> b1) -> b1)
flip ($) g              :: ((a -> b) -> b1) -> b1
(flip ($) g) . flip ($) :: a -> b
```

__Proof:__
```
(flip ($) g) . flip ($)                         =  (definition of "flip ($)")
((\x -> (\f -> f x)) g) . (\y -> (\f' -> f' y)) =  (eta reduction)
(\f -> f g) . (\y -> (\f' -> f' y))             =  (expansion of composition)
\y -> (\f' -> f' y) g                           =  (eta reduction)
\y -> g y                                       =  (function/lambda definition)
g  
```
&#9633;


[Back to Contents](#contents)

## Section 4.3 - Applicative Instances <a name="4.3"/>

### Ex. 4.3.1 - Applicative instance for Maybe <a name="4.3.1"/>

In [9]:
class MyApplicative f where
    pure'  :: a -> f a
    (<@>)  :: f (a -> b) -> f a -> f b
    
instance MyApplicative Maybe where
    pure' = Just
    Nothing <@> _ = Nothing
    _ <@> Nothing = Nothing
    Just g <@> Just a = Just (g a)
    

In [10]:
-- Test the Applicative laws.

-- Identity
-- pure' id <@> Nothing == Nothing
pure' id <@> Nothing -- The above causes the IHaskell kernel to hang, while working fine in ghci.
-- Curiously, even though the error message, below, calls for this,
-- when I attempt to use it I get an error saying the function
-- 'isNothing' is not in scope! (?!)
-- isNothing (pure' id <@> Nothing)
Just id <@> Just 3 == Just 3

-- Homomorphism
Just (+ 2) <@> Just 3 == Just ((+ 2) 3)

-- Interchange
Just (+ 2) <@> Just 3 == Just ($ 3) <@> Just (+ 2)

-- Composition
Just (+ 2) <@> (Just (+ 3) <@> Just 4) == Just (.) <@> Just (+ 2) <@> Just (+ 3) <@> Just 4

Nothing

True

True

True

True

[Back to Contents](#contents)

### Ex. 4.3.2 - Correct Applicative instance for ZipList <a name="4.3.2"/>

In [11]:
newtype ZipList a = ZipList { getZipList :: [a] } deriving (Eq)
 
instance MyApplicative ZipList where
  pure' x = ZipList (repeat x)
  (ZipList gs) <@> (ZipList xs) = ZipList (zipWith ($) gs xs)

-- Test the Applicative laws.

-- Identity
pure' id <@> ZipList ([1,2,3] :: [Int]) == ZipList [1,2,3]

-- Homomorphism
pure' (+ 2) <@> ZipList ([1,2,3] :: [Int]) == ZipList (map (+ 2) [1,2,3])

-- Interchange
ZipList (replicate 3 (+ 2)) <@> pure' 1 == pure' ($ 1) <@> ZipList (replicate 3 (+ 2))

-- Composition
ZipList (replicate 3 (+ 2)) <@> (ZipList (replicate 3 (+ 3)) <@> ZipList [1,2,3]) ==
    pure' (.) <@> ZipList (replicate 3 (+ 2)) <@> ZipList (replicate 3 (+ 3)) <@> ZipList [1,2,3]


True

True

True

True

[Back to Contents](#contents)

## Section 4.5 - Alternative Formulation (Monoidal) <a name="4.5"/>

### Ex. 4.5.1 - _pure_ and _<\*>_, via _unit_ and _(\*\*)_ <a name="4.5.1"/>

In [12]:
class Applicative f => Monoidal f where
  unit :: f ()
  unit  = pure ()
  
  (**) :: f a -> f b -> f (a,b)
  (**)  fa fb = pure (,) <*> fa <*> fb


In [13]:
class Monoidal f => MyApplicative f where
  pure'  :: a -> f a
  pure' x = fmap (const x) unit
  
  (<@>)    :: f (a -> b) -> f a -> f b
  fg <@> fx = fmap (uncurry ($)) (fg ** fx)

The fact that we have to `fmap` something over `unit` and `(**)`, in order to implement `pure` and `<*>`, suggests that the _Monoidal_ implementation is more general (since we could `fmap` something else, if that proved useful to some purpose).

[Back to Contents](#contents)

### Ex. 4.5.2 - Applicative instances providing inverses of Monoidal functions <a name="4.5.2"/>

Are there any Applicative instances for which there are also functions `f () -> ()` and `f (a,b) -> (f a, f b)`, satisfying some "reasonable" laws?

In [14]:
instance Monoidal Maybe

unit' :: Maybe () -> ()
unit' Nothing   = ()
unit' (Just ()) = ()

unjoin :: Maybe (a, b) -> (Maybe a, Maybe b)
unjoin Nothing       = (Nothing, Nothing)
unjoin (Just (x, y)) = (Just x, Just y)


**Laws:**

- Left Identity:  
  `first (arr unit') $ unjoin Just ((), x) ≅ Just x`
  
- Right Identity:  
  `second (arr unit') $ unjoin Just (x, ()) ≅ Just x`
  
- Inverse:  
  `unit' . unit == ()`

**Proofs:**

- Left Identity:  
  ```
  first (arr unit') $ unjoin Just ((), x) = (definition of unjoin)
  first (arr unit') (Just (), Just x)     = (application of "first (arr unit')")
  ((), Just x)                            ≅ (definition of isomorphism)
  Just x
  ```
  &#9633;
  
- Right Identity:  
  (Trivial, as per Left Identity.)
  
- Inverse:
  ```
  unit' . unit    = (definition of compose)
  unit' (unit)    = (definition of unit)
  unit' (Just ()) = (definition of unit')
  ()
  ```
  &#9633;


[Back to Contents](#contents)

### Ex. 4.5.3 - Equivalency of Applicative and Monoidal laws <a name="4.5.3"/>

(Tricky) Prove that given your implementations from the previous exercise, the usual Applicative laws and the Monoidal laws stated above are equivalent.

**Applicative Laws:**

- Identity:  
  `pure id <*> v = v`

- Homomorphism:  
  `pure f <*> pure x = pure (f x)`

- Interchange:  
  `u <*> pure y = pure ($ y) <*> u`

- Composition:  
  `u <*> (v <*> w) = pure (.) <*> u <*> v <*> w`

**Monoidal Laws:**

- Left Identity:  
  `unit ** v ≅ v`
  
- Right Identity:  
  `u ** unit ≅ u`

- Associativity:  
  `u ** (v ** w) ≅ (u ** v) ** w`

**Proofs:**

- Identity:

```
pure id <*> v                                    = (implementation of (<*>), via (**))
fmap (uncurry ($)) (pure id ** v)                = (implementation of pure, via unit)
fmap (uncurry ($)) ((fmap (const id) unit) ** v) = ()
```


[Back to Contents](#contents)